In [1]:
from tqdm import tqdm
import numpy as np
import scipy.io
from scipy.misc import imsave
import cv2 as cv
import os

In [2]:
cars_annos = scipy.io.loadmat('devkit/cars_train_annos')

In [3]:
annotations = cars_annos['annotations']
annotations = np.transpose(annotations)

In [4]:
annotations[0]

array([(array([[39]], dtype=uint8), array([[116]], dtype=uint8), array([[569]], dtype=uint16), array([[375]], dtype=uint16), array([[14]], dtype=uint8), array(['00001.jpg'], dtype='<U9'))],
      dtype=[('bbox_x1', 'O'), ('bbox_y1', 'O'), ('bbox_x2', 'O'), ('bbox_y2', 'O'), ('class', 'O'), ('fname', 'O')])

## Borrow from [https://github.com/foamliu/Car-Recognition/blob/master/pre-process.py](https://github.com/foamliu/Car-Recognition/blob/master/pre-process.py)

In [5]:
fnames = []
class_ids = []
bboxes = []
labels = []

for annotation in annotations:
    bbox_x1 = annotation[0][0][0][0]
    bbox_y1 = annotation[0][1][0][0]
    bbox_x2 = annotation[0][2][0][0]
    bbox_y2 = annotation[0][3][0][0]
    class_id = annotation[0][4][0][0]
    labels.append('%04d' % (class_id,))
    fname = annotation[0][5][0]
    bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
    class_ids.append(class_id)
    fnames.append(fname)

In [6]:
np.unique(class_ids).shape[0]

196

In [7]:
!rm -rf cars_train_crop

In [8]:
!mkdir cars_train_crop

In [9]:
src_folder = 'cars_train'
dst_folder = 'cars_train_crop'
margin = 16

for i in tqdm(range(len(fnames))):
    fname = fnames[i]
    label = labels[i]
    (x1, y1, x2, y2) = bboxes[i]

    src_path = os.path.join(src_folder, fname)
    src_image = cv.imread(src_path)
    height, width = src_image.shape[:2]
    
    x1 = max(0, x1 - margin)
    y1 = max(0, y1 - margin)
    x2 = min(x2 + margin, width)
    y2 = min(y2 + margin, height)
    
    dst_path = os.path.join(dst_folder, label + '_' + fname)
    crop_image = src_image[y1:y2, x1:x2]
    dst_img = cv.resize(src=crop_image, dsize=(height, width))
    imsave(dst_path + '.jpg', dst_img)

  0%|          | 0/8144 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
100%|██████████| 8144/8144 [02:45<00:00, 49.22it/s]
